# Script para la estimación de indicadores vinculados al **Índice de Desempeño de la Gestión en Inversión Pública (IDGIP)**

## 1. Librerías necesarias

In [1]:
# Estaslibrerías si no estan instaladas, se istalan con el comando pip install (ejemplo: pip install pandas) en la consola de Python
import pandas as pd #pip install pandas
import numpy as np #pip install numpy
import matplotlib.pyplot as plt #pip install matplotlib
import seaborn as sns #pip install seaborn
from pathlib import Path #pip install pathlib
from janitor import clean_names #pip install janitor
import getpass

# Para que las figuras se inserten en el notebook
%matplotlib inline

## 2. Rutas par importar la información

In [2]:
# Para poder correr el script en otras maquinas
user = getpass.getuser() # Capturo el usuario local / Ejemplo "Llan_", este es el nombre de mi usuario que inicie sesion en la pc
user=user.upper()

if    user== "LLAN_": # mi usuario
      ruta = Path(r"D:\PROPUESTAS NELSON\INDICADOR\ESTIMACIÓN")
elif  user== "Otro usuario": # Se debería colocar el nombre del usuario
      ruta = Path(r"D:\PROPUESTAS NELSON\INDICADOR\ESTIMACIÓN") # se deberpia poner la ruta del otro usario

In [3]:
# Defino la ruta donde se encuentra mi "input"
ruta_input=ruta / 'Insumos'  

# Defino la ruta donde exporare mi "Output"
ruta_output=ruta / 'Output'  

## 3. Cargando la información (En archivos Excel)


### 3.1 Lista de inversiones totales (rep_inv_tocache_leoncio_prado)

In [4]:
# pip install openpyxl
data_inversiones = pd.read_excel(ruta_input / 'rep_inv_tocache_leoncio_prado.xlsx', sheet_name="SQL Results")

In [5]:
# Observamos la lista de variables del "Dataframe"
data_inversiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8337 entries, 0 to 8336
Data columns (total 100 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              8337 non-null   int64         
 1   CODIGO_UNICO            7604 non-null   float64       
 2   CODIGO_SNIP             8337 non-null   int64         
 3   NOMBRE_INVERSION        8337 non-null   object        
 4   NIVEL                   8337 non-null   object        
 5   SECTOR                  8337 non-null   object        
 6   ENTIDAD                 8299 non-null   object        
 7   OPMI                    2361 non-null   object        
 8   RESPONSABLE_OPMI        2361 non-null   object        
 9   UF                      8337 non-null   object        
 10  RESP_NOMBRE_UF          8337 non-null   object        
 11  UEI                     2365 non-null   object        
 12  RESPONSABLE_UEI         2365 non-null   object 

In [6]:
# Nos concetraremos en inversiones de "Gobierno Local (GL)" y de las provincias de Tocache y Leoncio Prado y sus distritos, Luego cuando se tenga que exportar
# se tomara en cuenta solo los del ámbito de estudio.

data_inversiones.groupby(['NIVEL', 'ENTIDAD']).size()

NIVEL  ENTIDAD                                                                                    
GL     MUNICIPALIDAD DISTRITAL DE CASTILLO GRANDE                                                      76
       MUNICIPALIDAD DISTRITAL DE DANIEL ALOMIA ROBLES                                                431
       MUNICIPALIDAD DISTRITAL DE HERMILIO VALDIZAN                                                   328
       MUNICIPALIDAD DISTRITAL DE HUANCA HUANCA                                                         1
       MUNICIPALIDAD DISTRITAL DE JIRCAN                                                               20
       MUNICIPALIDAD DISTRITAL DE JOSE CRESPO Y CASTILLO                                              631
       MUNICIPALIDAD DISTRITAL DE LUYANDO                                                             480
       MUNICIPALIDAD DISTRITAL DE MARIANO DAMASO BERAUN                                               343
       MUNICIPALIDAD DISTRITAL DE NUEVO PROGRESO     

In [7]:
# Nos concetramos en inversiones de Gobiernos Locales (Es decir las municipaliadades)
data_inversiones =data_inversiones.loc[(data_inversiones['NIVEL']=="GL")]

Creando un ID para reconocer la municipalidad pronvincia y distrital de Tocache

In [8]:
data_inversiones.loc[((data_inversiones.ENTIDAD=="MUNICIPALIDAD PROVINCIAL DE TOCACHE")) , 'id_ambito']="ProvTocache"

In [9]:
data_inversiones.loc[((data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE NUEVO PROGRESO") | 
(data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE POLVORA") | (data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE SHUNTE")
| (data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE UCHIZA")) , 'id_ambito']="DistTocache"

Creando un ID para reconocer la municipalidad pronvincia y distrital de Leoncio Prado

In [10]:
data_inversiones.loc[((data_inversiones.ENTIDAD=="MUNICIPALIDAD PROVINCIAL DE LEONCIO PRADO")) , 'id_ambito']="ProvLeoncioPrado"

In [11]:
data_inversiones.loc[((data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE CASTILLO GRANDE") | 
(data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE DANIEL ALOMIA ROBLES") | (data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE HERMILIO VALDIZAN")
| (data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE JOSE CRESPO Y CASTILLO") | (data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE LUYANDO")
| (data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE MARIANO DAMASO BERAUN") | (data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE PUCAYACU")
| (data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE PUEBLO NUEVO-HUANUCO") | (data_inversiones.ENTIDAD=="MUNICIPALIDAD DISTRITAL DE SANTO DOMINGO DE ANDA")) , 'id_ambito']="DistLeoncioPrado"

Filtramos inversiones de Leoncio Prado y Tocache (Tanto provinciales como distritales)

In [12]:
data_inversiones=data_inversiones.loc[(data_inversiones['id_ambito']=="ProvTocache") | (data_inversiones['id_ambito']=="DistTocache") 
| (data_inversiones['id_ambito']=="ProvLeoncioPrado") | (data_inversiones['id_ambito']=="DistLeoncioPrado")]

In [13]:
# Remplazando missings
data_inversiones.loc[(data_inversiones['MTO_PMI_2019'].isnull()),'MTO_PMI_2019']=0
data_inversiones.loc[(data_inversiones['MTO_PMI_2020'].isnull()),'MTO_PMI_2020']=0
data_inversiones.loc[(data_inversiones['MTO_PMI_2021'].isnull()),'MTO_PMI_2021']=0
data_inversiones.loc[(data_inversiones['MTO_PMI_2022'].isnull()),'MTO_PMI_2022']=0

data_inversiones.loc[(data_inversiones['PIM_2019'].isnull()),'PIM_2019']=0
data_inversiones.loc[(data_inversiones['PIM_2020'].isnull()),'PIM_2020']=0
data_inversiones.loc[(data_inversiones['PIM_2021'].isnull()),'PIM_2021']=0
data_inversiones.loc[(data_inversiones['PIM_2022'].isnull()),'PIM_2022']=0

data_inversiones.loc[(data_inversiones['DEVEN_2019'].isnull()),'DEVEN_2019']=0
data_inversiones.loc[(data_inversiones['DEVEN_2020'].isnull()),'DEVEN_2020']=0
data_inversiones.loc[(data_inversiones['DEVEN_2021'].isnull()),'DEVEN_2021']=0
data_inversiones.loc[(data_inversiones['DEVEN_2022'].isnull()),'DEVEN_2022']=0

In [14]:
# Año de viabilidad de las inversiones
data_inversiones['año']=data_inversiones['FECHA_VIAB'].dt.year    
data_inversiones['mes']=data_inversiones['FECHA_VIAB'].dt.month    
data_inversiones['dia']=data_inversiones['FECHA_VIAB'].dt.day    

# Total de días de registro
#data_inversiones['dia_viable_total']=data_inversiones['año']*365 + data_inversiones['mes']*30 + data_inversiones['FECHA_VIAB'].dt.day    

data_inversiones['año_reg']=data_inversiones['FECHA_VIAB'].dt.year    
data_inversiones['mes_reg']=data_inversiones['FECHA_VIAB'].dt.month    
data_inversiones['dia_reg']=data_inversiones['FECHA_VIAB'].dt.day 


In [15]:
#Formato de fecha sin considerar horas
data_inversiones['FECHA_VIAB'] = pd.to_datetime(data_inversiones['FECHA_VIAB']).dt.normalize() 
data_inversiones['FECHA_REGI'] = pd.to_datetime(data_inversiones['FECHA_REGI']).dt.normalize() 

In [16]:
# Extraendo los tipos de problemas registrados
data_inversiones['PROBLEMA'] = (data_inversiones['DES_ULT_PROBLEMA'].str.findall(pat = '\((.*?)\)')).str[0]

<>:2: DeprecationWarning: invalid escape sequence '\('
<>:2: DeprecationWarning: invalid escape sequence '\('
C:\Users\llan_\AppData\Local\Temp\ipykernel_1472\2784150869.py:2: DeprecationWarning: invalid escape sequence '\('
  data_inversiones['PROBLEMA'] = (data_inversiones['DES_ULT_PROBLEMA'].str.findall(pat = '\((.*?)\)')).str[0]


In [17]:
#Solo se consideran Proyectos de Inversion e IOARR
data_inversiones=data_inversiones.loc[~((data_inversiones['DES_TIPO_FORMATO']=="PROGRAMA DE INVERSION") | (data_inversiones['DES_TIPO_FORMATO']=="PROGRAMA (SNIP)"))]

In [18]:
#data_fake=data_inversiones[['CODIGO_UNICO','CODIGO_SNIP','ULT_PROBLEMA','DES_ULT_PROBLEMA','ACC_PROBLEMA','PROBLEMA']]
#data_fake.to_excel(ruta_output / 'fake.xlsx', sheet_name='fake' , index= False)

In [19]:
#data_inversiones.groupby(['año']).size()

In [20]:
#data_inversiones['año'].info()

In [21]:
# Exportando la base
data_inversiones.to_excel(ruta_output / 'base_inversiones_intermedia.xlsx', sheet_name='inversiones' , index= False)

### 3.2 Base Ejecución de inversiones (rep_deveng_mes)

In [22]:
data_ejecucion_inv = pd.read_excel(ruta_input / 'rep_deveng_mes.xlsx', sheet_name="SQL Results")

In [23]:
#string la fecha
data_ejecucion_inv.loc[(data_ejecucion_inv['MES_EJE'].isnull()),'MES_EJE']=0
data_ejecucion_inv['MES_EJE']=data_ejecucion_inv['MES_EJE'].astype(int)
data_ejecucion_inv['MES_EJE']=data_ejecucion_inv['MES_EJE'].astype(str)
data_ejecucion_inv['ANO_EJE']=data_ejecucion_inv['ANO_EJE'].astype(str)

In [24]:
#crendo el string de fecha
data_ejecucion_inv['FECHA_EJEC']="01" + "/" + data_ejecucion_inv['MES_EJE'] + "/"  + data_ejecucion_inv['ANO_EJE'] 
data_ejecucion_inv['FECHA_EJEC'] = pd.to_datetime(data_ejecucion_inv['FECHA_EJEC'],errors="coerce" )

In [25]:
#data_ejecucion_inv.to_excel(ruta_output / 'fake_2.xlsx', sheet_name='inversiones' , index= False)

### 3.3 Base aprobación de ET o DE (rep_et_de_aprob)

In [26]:
data_et_inv = pd.read_excel(ruta_input / 'rep_et_de_aprob.xlsx', sheet_name="SQL Results")

In [27]:
#Fecha de parobación de expediente
data_et_inv['FEC_APROBACION'] = pd.to_datetime(data_et_inv['FEC_APROBACION'],errors="coerce" ).dt.normalize() 
data_et_inv_por_cod=data_et_inv[['CODIGO_UNICO','FEC_APROBACION']].groupby(by=['CODIGO_UNICO'], as_index=False).max()

### 3.4 Base de cierre de brechas (rep_inv_brechas)

In [28]:
data_cierre_brech_inv = pd.read_excel(ruta_input / 'rep_inv_brechas.xlsx', sheet_name="SQL Results")

### 3.5 Base SEACE de inversiones (repinv_seace)

In [29]:
data_seace_inv = pd.read_excel(ruta_input / 'repinv_seace.xlsx', sheet_name="SQL Results")

### 3.6 Base inversiones formato 8 (rep_f8_et_activid, rep_f8_et_detalle, rep_f8_et_mto_cronog)

In [30]:
data_f8_activ_inv = pd.read_excel(ruta_input / 'rep_f8_et_activid.xlsx', sheet_name="SQL Results")

In [31]:
data_f8_detall_inv = pd.read_excel(ruta_input / 'rep_f8_et_detalle.xlsx', sheet_name="SQL Results")

In [32]:
#Remplazo algunas inconsistencias que puedan existir en la base
data_f8_detall_inv.loc[data_f8_detall_inv['COSTO_INVERSION'].isnull(),'COSTO_INVERSION']=0
data_f8_detall_inv.loc[data_f8_detall_inv['COSTO_INVERSION']<0,'COSTO_INVERSION']=0

In [33]:
data_f8_cronog_inv = pd.read_excel(ruta_input / 'rep_f8_et_mto_cronog.xlsx', sheet_name="SQL Results")

## 4. Estimación de indicadores

### Indicador **PRG01**: Porcentaje de inversiones no previstas de la cartera

In [34]:
# Exportando la base
writer = pd.ExcelWriter(ruta_output / 'base_por_indicador.xlsx')

In [35]:
# Inversiones de cartera
data_inversiones.loc[((data_inversiones.REG_CART_PMI_2019=="SI") ) , 'PMI_1']=1
data_inversiones.loc[((data_inversiones.REG_CART_PMI_2020=="SI") ) , 'PMI_2']=1
data_inversiones.loc[((data_inversiones.REG_CART_PMI_2021=="SI") ) , 'PMI_3']=1
data_inversiones.loc[((data_inversiones.REG_CART_PMI_2022=="SI")  ) , 'PMI_4']=1

# Inversiones que no son "No previstas"
data_inversiones.loc[((data_inversiones.REG_CART_PMI_2019=="SI") & (data_inversiones.NO_PREVISTA_2019=="NO") ) , 'PMI_1_PREVISTAS']=1
data_inversiones.loc[((data_inversiones.REG_CART_PMI_2020=="SI") & (data_inversiones.NO_PREVISTA_2020=="NO") ) , 'PMI_2_PREVISTAS']=1
data_inversiones.loc[((data_inversiones.REG_CART_PMI_2021=="SI") & (data_inversiones.NO_PREVISTA_2021=="NO") ) , 'PMI_3_PREVISTAS']=1
data_inversiones.loc[((data_inversiones.REG_CART_PMI_2022=="SI") & (data_inversiones.NO_PREVISTA_2022=="NO") ) , 'PMI_4_PREVISTAS']=1

In [36]:
# Para la Municipalidad del ámbito de intervención
#data_prov_tocache =data_inversiones.loc[(data_inversiones['id_ambito']=='ProvTocache')]
data_prov=data_inversiones.drop_duplicates(['CODIGO_SNIP'], keep='first') 

#Data intermedía
data_prov_ind1=data_prov.copy()
data_prov_ind1=data_prov_ind1[['CODIGO_UNICO','CODIGO_SNIP','NOMBRE_INVERSION', 'NIVEL', 'SECTOR','ENTIDAD','id_ambito','FECHA_REGI', 'FECHA_VIAB',
'MONTO_VIABLE', 'COSTO_ACTUALIZADO','FUNCION','PROGRAMA','SUBPROGRAM', 'ESTADO', 'SITUACION', 'ULT_ESTU', 'ESTD_ESTU',
'DEVENGADO_ACUMULADO', 'DEV_AÑO_ACTUAL', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'UBIGEO', 'NIVREQVIAB', 'MARCO', 
'DES_TIPO_FORMATO', 'DES_TIPOLOGIA', 'ET_REGISTRADO', 'MTO_EXP_TCO', 'REG_CART_PMI_2023', 'REG_CART_PMI_2022', 
'REG_CART_PMI_2021', 'REG_CART_PMI_2020', 'REG_CART_PMI_2019', 'MTO_PMI_2023', 'MTO_PMI_2022', 'MTO_PMI_2021', 'MTO_PMI_2020',
'MTO_PMI_2019', 'PIM_2023', 'PIM_2022', 'PIM_2021', 'PIM_2020', 'PIM_2019', 'DEVEN_2023', 'DEVEN_2022', 'DEVEN_2021', 'DEVEN_2020'
, 'DEVEN_2019', 'NO_PREVISTA_2023', 'NO_PREVISTA_2022', 'NO_PREVISTA_2021', 'NO_PREVISTA_2020', 'NO_PREVISTA_2019'
, 'ULT_PROBLEMA', 'DES_ULT_PROBLEMA', 'ACC_PROBLEMA', 'NUM_POSTORES', 'INFORME_CIERRE', 'FEC_CIERRE', 'DES_CIERRE','TIENE_F9'
, 'FEC_REG_F9', 'FEC_ULT_MODIF_F9', 'SECCION_F9', 'ESTA_CULMINADA', 'ESTA_FUNCIONAMIENTO', 'FEC_LIQUIDACION','FEC_PRELIQUIDACION',
'LIQUIDACION_OFICIO','PMI_1', 'PMI_2', 'PMI_3','PMI_4','PMI_1_PREVISTAS','PMI_2_PREVISTAS','PMI_3_PREVISTAS',
'PMI_4_PREVISTAS']]

data_prov=data_prov[["PMI_2","PMI_2_PREVISTAS","PMI_3","PMI_3_PREVISTAS","PMI_4","PMI_4_PREVISTAS","ENTIDAD","id_ambito"]].groupby(by=['ENTIDAD', 'id_ambito'],as_index=False).sum()

#data_prov['PRG01_1']=data_prov['PMI_1_PREVISTAS']/data_prov['PMI_1']
data_prov['PRG01_2']=data_prov['PMI_2_PREVISTAS']/data_prov['PMI_2']
data_prov['PRG01_3']=data_prov['PMI_3_PREVISTAS']/data_prov['PMI_3']
data_prov['PRG01_4']=data_prov['PMI_4_PREVISTAS']/data_prov['PMI_4']

data_prov['PRG01_PROM']=(data_prov['PRG01_2']+data_prov['PRG01_3']+data_prov['PRG01_4'])/(3)

# Exportando la base
#writer = pd.ExcelWriter(ruta_output / 'indicadores_1.xlsx')
data_prov.to_excel(writer, sheet_name='PRG01' , index= False)
data_prov_1=data_prov.copy()
#writer.save()
#writer.close()

### Indicador **PRG02**: Consistencia del monto programado en el PMI respecto al PIM || **PRG03**:Consistencia del monto programado en el PMI respecto al devengado

In [37]:
# Inversiones no duplicadas y con codigo único
data_prov=data_inversiones.drop_duplicates(['CODIGO_SNIP'], keep='first') 
data_prov =data_prov.loc[~(data_prov['CODIGO_UNICO'].isnull())]

# Generamos el monto programado, pim y devengado de la cartera
data_prov['MONTO_PROGRAMADO'] = 0        
data_prov.loc[data_prov['REG_CART_PMI_2020']=="SI", 'MONTO_PROGRAMADO']=data_prov['MTO_PMI_2020']+data_prov['MTO_PMI_2021']+data_prov['MTO_PMI_2022']
data_prov['MONTO_PIM'] =0
data_prov.loc[data_prov['REG_CART_PMI_2020']=="SI", 'MONTO_PIM']=data_prov['PIM_2020']+data_prov['PIM_2021']+data_prov['PIM_2022']
data_prov['MONTO_DEVENGADO'] = 0
data_prov.loc[data_prov['REG_CART_PMI_2020']=="SI", 'MONTO_DEVENGADO']=data_prov['DEVEN_2020']+data_prov['DEVEN_2021']+data_prov['DEVEN_2022']

data_prov_ind2_3=data_prov.copy()
data_prov_ind2_3=data_prov_ind2_3[['CODIGO_SNIP','MONTO_PROGRAMADO', 'MONTO_PIM','MONTO_DEVENGADO']]

data_prov=data_prov[["MONTO_PROGRAMADO","MONTO_PIM","MONTO_DEVENGADO","ENTIDAD","id_ambito"]].groupby(by=['ENTIDAD', 'id_ambito'],as_index=False).sum()

# Estimacion de los indicadores
data_prov['PRG02']=data_prov['MONTO_PIM']/data_prov['MONTO_PROGRAMADO']
data_prov.loc[data_prov['PRG02']>1, 'PRG02']=0  # Si es mayor a cero indicamos que es cero

data_prov['PRG03']=data_prov['MONTO_DEVENGADO']/data_prov['MONTO_PROGRAMADO']
data_prov.loc[data_prov['PRG03']>1, 'PRG03']=0  # Si es mayor a cero indicamos que es cero

# Exportando la base
#writer = pd.ExcelWriter(ruta_output / 'indicadores_2.xlsx')
data_prov.to_excel(writer, sheet_name='PRG02_PRG03' , index= False)
data_prov_2_3=data_prov.copy()
#writer.save()
#writer.close()

### Indicador PRG04: Porcentaje de Inversiones con registro en Cierre de brecha

In [38]:
# Identificando la cantidad de inversiones totales entre el 2012-2022
data_prov=data_inversiones.loc[(data_inversiones['SITUACION']=="VIABLE") | (data_inversiones['SITUACION']=="APROBADO")]     
data_prov=data_prov.drop_duplicates(['CODIGO_SNIP'], keep='first')
#data_prov=data_prov.loc[~(data_prov['CODIGO_UNICO'].isnull())]  # Inversiones que cuentan con código único
data_prov["id_inversion"]=1

#Inversiones entre el 2012 y 2022
data_prov=data_prov.loc[(data_prov['año']>=2012) & (data_prov['año']<=2022)]  #Inversiones entre el 2017 y 2022
data_prov=data_prov.loc[data_prov['MARCO']=='INVIERTE']  #Inversiones en el marco del "INVIERTE"

In [39]:
# Construindo las inversiones que cuentan con valor de cierre de brecha
data_cierre_brech_inv["id_brecha"]=0
data_cierre_brech_inv.loc[data_cierre_brech_inv.VAL_CIERRE_BRECHA>0, 'id_brecha']=1

data_cierre_brech_inv=data_cierre_brech_inv[["id_brecha","CODIGO_UNICO"]].groupby(by=['CODIGO_UNICO'],as_index=False).sum()
data_cierre_brech_inv.loc[data_cierre_brech_inv.id_brecha>0, 'id_brecha']=1

In [40]:
#Join
data_prov_union=data_prov.merge(right = data_cierre_brech_inv, how="left", on=["CODIGO_UNICO"] )
data_prov_union.loc[(data_prov_union['id_brecha'].isnull()),'id_brecha']=0 

#import previo
data_prov_ind4=data_prov_union.copy()
data_prov_ind4=data_prov_ind4[['CODIGO_SNIP','id_brecha','id_inversion']]

data_prov_union=data_prov_union[["id_ambito","ENTIDAD","id_brecha","id_inversion"]].groupby(by=['ENTIDAD', 'id_ambito'],as_index=False).sum()

# Estimacion de los indicadores
data_prov_union['PRG04']=data_prov_union['id_brecha']/data_prov_union['id_inversion']

In [41]:
#Exportando resultado
#writer = pd.ExcelWriter(ruta_output / 'indicadores_4.xlsx')
data_prov_union.to_excel(writer, sheet_name='PRG04' , index= False)
data_prov_union_4=data_prov_union.copy()
#writer.save()
#writer.close()

### Indicador **FORM01**: Razonabilidad de la estimación del costo de inversión en la formulación, **FORM02**: Razonabilidad del periodo de formulación de la inversión, **EJEC01**:N° de Inversiones viables cuyo devengado es mayor que cero, **EJEC02**:Devengado acumulado de las inversiones respecto de sus costos de inversión a nivel de viabilidad, **EJEC03**: Porcentaje de inversiones en las que la elaboración del ET toma menos de un año, **EJEC04**:Avance en la ejecución financiera de las inversiones declaradas viables, **EJEC05**:Porcentaje de inversiones en ejecución que no tienen observaciones de la Contraloría General de la República  (CGR).  

In [42]:
# Identificando la cantidad de inversiones totales entre el 2012-2022
data_prov=data_inversiones.loc[(data_inversiones['SITUACION']=="VIABLE") | (data_inversiones['SITUACION']=="APROBADO")]     
data_prov=data_prov.drop_duplicates(['CODIGO_SNIP'], keep='first')
#data_prov=data_prov.loc[~(data_prov['CODIGO_UNICO'].isnull())]  # Inversiones que cuentan con código único
#data_prov["id_inversion"]=1

#Inversiones entre el 2012 y 2022
data_prov=data_prov.loc[(data_prov['año']>=2012) & (data_prov['año']<=2022)]  #Inversiones entre el 2012 y 2022

In [43]:
#Diferenciando la fecha de viabilidad y registro
data_prov['DIF_FECHA']= data_prov['FECHA_VIAB'] - data_prov['FECHA_REGI']
#data_prov['DIF_FECHA']= data_prov['DIF_FECHA'].dt.strftime
#data_prov['DIF_FECHA']= data_prov['DIF_FECHA'].astype(float)

In [44]:
# Estimando los días
data_prov['DIF_FECHA']= data_prov['DIF_FECHA'].dt.total_seconds()/(60*24*60)
data_prov['DIF_FECHA']= data_prov['DIF_FECHA'].astype(str)
data_prov['DIF_FECHA']= data_prov['DIF_FECHA'].astype(float)

In [45]:
#Identificando el tipo de formato:DES_TIPO_FORMATO
data_prov["FORMATO"]="Otros"
data_prov.loc[(data_prov['ULT_ESTU']=="FICHA TÉCNICA SIMPLIFICADA"),"FORMATO"]="FTS" 
data_prov.loc[(data_prov['ULT_ESTU']=="FICHA TÉCNICA ESTANDAR"),"FORMATO"]="FTE" 
data_prov.loc[(data_prov['ULT_ESTU']=="FICHA TECNICA DE  BAJA Y MEDIANA COMPLEJIDAD"),"FORMATO"]="FTBMC" 
data_prov.loc[(data_prov['ULT_ESTU']=="PERFIL"),"FORMATO"]="PERFIL" 
data_prov.loc[(data_prov['DES_TIPO_FORMATO']=="INVERSIONES IOARR"),"FORMATO"]="IOARR" 
data_prov.loc[(data_prov['DES_TIPO_FORMATO']=="INTERVENCIONES IRI"),"FORMATO"]="IRI" 

In [46]:
#Identificando inversiones viables entre 2012-2022 con devengado acumulado mayor acero
data_prov['id_FORM01_D']=0
data_prov.loc[(data_prov['DEVENGADO_ACUMULADO']>0),"id_FORM01_D"]=1

#Identificando inversiones viables entre 2012-2022
data_prov['id_FORM02_D']=1
#data_prov.loc[(data_prov['DEVENGADO_ACUMULADO']>0),"id_FORM02_D"]=1

#Identificando inversiones viables entre 2012-2022
data_prov['id_EJEC01_D']=1

#Identificando inversiones viables entre 2012-2022 con devengado acumulado
data_prov['id_EJEC02_D']=data_prov['COSTO_ACTUALIZADO']
#Identificando inversiones y su PIM (2022)
data_prov['id_EJEC04_D']=data_prov['PIM_2022']

#Identificando inversiones y su PIM (2022)
data_prov['id_EJEC05_D']=0
data_prov.loc[(data_prov['DEVENGADO_ACUMULADO']>0),"id_EJEC05_D"]=1


In [47]:
#Identifico el todal de inversiones viables y que cuenten con devengado mayor a ceron entre 2012-2022, donde:  DEVENGADO_ACUMULADO<=MONTO_VIABLE 
data_prov['id_FORM01_N']=0
data_prov.loc[((data_prov['DEVENGADO_ACUMULADO']>0) & (data_prov['DEVENGADO_ACUMULADO']<=data_prov['MONTO_VIABLE'])),"id_FORM01_N"]=1


# Inversiones que cumplen el siguiente criterio:
'''
El tiempo esperado de declaración de viabilidad de una inversión es fijado a través del tipo de formato al que corresponde. De tal forma:
-Si se trata de una inversión IOARR, el periodo debería ser entre 15 días y 1 mes.
-Si se trata de una intervención IRI, el periodo debería ser entre 0 días y 6 meses.
-Si se trata de una Ficha Técnica Simplificada, el periodo debería ser entre 1 mes y 2 meses.
-Si se trata de una Ficha Técnica Estándar, el periodo debería ser entre 3 meses y 6 meses.
-Si se trata de una Ficha Técnica Baja y Mediana Complejidad, el periodo debería ser entre 3 meses y 6 meses.
-Si se trata de un Perfil, el periodo debería ser entre 4 meses y 12 meses.
-Si se trata de Otros formatos, el periodo debería ser entre 4 meses y 12 meses.
'''

# Identificado la razonabilidad entre la fecha de viabilidad y fecha de registro
data_prov['id_FORM02_N']=0
data_prov.loc[(((data_prov['DIF_FECHA']>0) & (data_prov['DIF_FECHA']<=30)) & (data_prov['FORMATO']=="IOARR")),"id_FORM02_N"]=1
data_prov.loc[(((data_prov['DIF_FECHA']>0) & (data_prov['DIF_FECHA']<=30*6)) & (data_prov['FORMATO']=="IRI")),"id_FORM02_N"]=1
data_prov.loc[(((data_prov['DIF_FECHA']>=1*30) & (data_prov['DIF_FECHA']<=2*30)) & (data_prov['FORMATO']=="FTS")),"id_FORM02_N"]=1
data_prov.loc[(((data_prov['DIF_FECHA']>=3*30) & (data_prov['DIF_FECHA']<=6*30)) & (data_prov['FORMATO']=="FTE")),"id_FORM02_N"]=1
data_prov.loc[(((data_prov['DIF_FECHA']>=3*30) & (data_prov['DIF_FECHA']<=6*30)) & (data_prov['FORMATO']=="FTBMC")),"id_FORM02_N"]=1
data_prov.loc[(((data_prov['DIF_FECHA']>=4*30) & (data_prov['DIF_FECHA']<=12*30)) & (data_prov['FORMATO']=="PERFIL")),"id_FORM02_N"]=1
data_prov.loc[(((data_prov['DIF_FECHA']>=4*30) & (data_prov['DIF_FECHA']<=12*30)) & (data_prov['FORMATO']=="Otros")),"id_FORM02_N"]=1

#Identificando inversiones viables entre 2012-2022 con devengado acumulado mayor acero
data_prov['id_EJEC01_N']=0
data_prov.loc[(data_prov['DEVENGADO_ACUMULADO']>0),"id_EJEC01_N"]=1

#Identificando inversiones viables entre 2012-2022 con devengado acumulado
data_prov['id_EJEC02_N']=data_prov['DEVENGADO_ACUMULADO']

#Identificando inversiones y su devengado (2022)
data_prov['id_EJEC04_N']=data_prov['DEVEN_2022']

#Identificando inversiones que no presentan observaciones de la contraloría
data_prov['id_EJEC05_N']=1
data_prov.loc[((data_prov['PROBLEMA']=='SUPERVISOR/INSPECTOR') | (data_prov['PROBLEMA']=='RIESGO NO IDENTIFICADO')|(data_prov['PROBLEMA']=='RESOLUCIÓN DE CONVENIO')
|(data_prov['PROBLEMA']=='RESOLUCIÓN DE CONTRATO')|(data_prov['PROBLEMA']=='RECEPCIÓN OBSERVADA')|(data_prov['PROBLEMA']=='PERMISOS Y LICENCIAS')
|(data_prov['PROBLEMA']=='OTRO')|(data_prov['PROBLEMA']=='OTRAS AUTORIZACIONES')|(data_prov['PROBLEMA']=='MODIFICACIONES EN CALENDARIO DE PROCEDIMIENTO DE SELECCIÓN')
|(data_prov['PROBLEMA']=='INTERFERENCIAS')|(data_prov['PROBLEMA']=='FALTA DE RECURSOS FINANCIEROS')|(data_prov['PROBLEMA']=='FALTA DE DISPONIBILIDAD DE TERRENO')
|(data_prov['PROBLEMA']=='DISCREPANCIAS (VALORIZACIÓN OBSERVADA, ETC.)') |(data_prov['PROBLEMA']=='ATRASOS Y/O PARALIZACIONES')|(data_prov['PROBLEMA']=='ARBITRAJE')),"id_EJEC05_N"]=0

data_prov.loc[(data_prov['DEVENGADO_ACUMULADO']<=0), 'id_EJEC05_N']=0 

data_prov_ind1_2_1_2_4_5=data_prov.copy()
data_prov_ind1_2_1_2_4_5=data_prov_ind1_2_1_2_4_5[['CODIGO_SNIP',"id_FORM01_N","id_FORM01_D","id_FORM02_D",'id_FORM02_N','id_EJEC01_N','id_EJEC01_D','id_EJEC02_N','id_EJEC02_D',
'id_EJEC04_N','id_EJEC04_D','id_EJEC05_N','id_EJEC05_D']]

In [48]:
# Collapsando
data_prov=data_prov[["id_ambito","ENTIDAD","id_FORM01_N","id_FORM01_D","id_FORM02_D",'id_FORM02_N','id_EJEC01_N','id_EJEC01_D','id_EJEC02_N','id_EJEC02_D',
'id_EJEC04_N','id_EJEC04_D','id_EJEC05_N','id_EJEC05_D']].groupby(by=['ENTIDAD', 'id_ambito'],as_index=False).sum()

In [49]:
# Estimacion de los indicadores
data_prov['FORM01']=data_prov['id_FORM01_N']/data_prov['id_FORM01_D']
data_prov['FORM02']=data_prov['id_FORM02_N']/data_prov['id_FORM02_D']
data_prov['EJEC01']=data_prov['id_EJEC01_N']/data_prov['id_EJEC01_D']
data_prov['EJEC02']=data_prov['id_EJEC02_N']/data_prov['id_EJEC02_D']
data_prov['EJEC04']=data_prov['id_EJEC04_N']/data_prov['id_EJEC04_D']
data_prov['EJEC05']=data_prov['id_EJEC05_N']/data_prov['id_EJEC05_D']


In [50]:
#Exportando resultado
#writer = pd.ExcelWriter(ruta_output / 'indicadores_5.xlsx')
data_prov.to_excel(writer, sheet_name='FORM01_02_EJEC01_02_03_04_05' , index= False)
data_prov_1_2_1_2_4_5=data_prov.copy()
#writer.save()
#writer.close()

### Indicador EJEC03: Porcentaje de inversiones en las que la elaboración del ET toma menos de un año 

In [51]:
# Identificando la cantidad de inversiones viables totales entre el 2012-2022
data_prov=data_inversiones.loc[(data_inversiones['SITUACION']=="VIABLE")  | (data_inversiones['SITUACION']=="APROBADO")]     
data_prov=data_prov.drop_duplicates(['CODIGO_SNIP'], keep='first')

#Inversiones entre el 2012 y 2022
data_prov=data_prov.loc[(data_prov['año']>=2012) & (data_prov['año']<=2022)]  #Inversiones entre el 2012 y 2022

In [52]:
# join con la base fecha de expedientes tecnicos
data_et_inv_union=data_prov.merge(right=data_et_inv_por_cod, how='left',on=["CODIGO_UNICO"])


In [53]:
#diferenciando la fecha de aprobacion del ET (FEC_APROBACION) y la fecha de viabilidad de la inversión (FECHA_VIAB)
data_et_inv_union['DIF_FECHA_ET']=data_et_inv_union['FEC_APROBACION'] - data_et_inv_union['FECHA_VIAB']

# Estimando los días
data_et_inv_union['DIF_FECHA_ET']= data_et_inv_union['DIF_FECHA_ET'].dt.total_seconds()/(60*24*60)
data_et_inv_union['DIF_FECHA_ET']= data_et_inv_union['DIF_FECHA_ET'].astype(str)
data_et_inv_union['DIF_FECHA_ET']= data_et_inv_union['DIF_FECHA_ET'].astype(float)


In [54]:
#Creando el indicador

#Numerador
data_et_inv_union['id_EJEC03_N']=0
data_et_inv_union.loc[(data_et_inv_union['DIF_FECHA_ET']>=0) & (data_et_inv_union['DIF_FECHA_ET']<=365),'id_EJEC03_N' ]=1

#Denominador
data_et_inv_union['id_EJEC03_D']=1  #Todas las inversiones de esta base son viables

#Base preliminar
data_et_inv_union_ind3=data_et_inv_union.copy()
data_et_inv_union_ind3=data_et_inv_union_ind3[['CODIGO_SNIP',"id_EJEC03_N","id_EJEC03_D"]]

In [55]:
#Collapsando
data_et_inv_collapse=data_et_inv_union[["id_ambito","ENTIDAD","id_EJEC03_N","id_EJEC03_D"]].groupby(by=['ENTIDAD', 'id_ambito'],as_index=False).sum()

In [56]:
#Indicador
data_et_inv_collapse['EJEC03']=data_et_inv_collapse['id_EJEC03_N']/data_et_inv_collapse['id_EJEC03_D']

In [57]:
#Exportando resultado
#writer = pd.ExcelWriter(ruta_output / 'indicadores_6.xlsx')
data_et_inv_collapse.to_excel(writer, sheet_name='EJEC03' , index= False)
data_et_inv_collapse_3=data_et_inv_collapse.copy()
#writer.save()
#writer.close()

### **EJEC06**: Razonabilidad del periodo de ejecución de una inversión

In [58]:
# Identificando la cantidad de inversiones viables totales entre el 2012-2022
data_prov=data_inversiones.loc[(data_inversiones['SITUACION']=="VIABLE")]      #| (data_inversiones['SITUACION']=="APROBADO")
data_prov=data_prov.drop_duplicates(['CODIGO_SNIP'], keep='first')

#Inversiones entre el 2012 y 2022
data_prov=data_prov.loc[(data_prov['año']>=2012) & (data_prov['año']<=2022)]  #Inversiones entre el 2012 y 2022

In [59]:
# collapse
data_ejecucion_inv_1= data_ejecucion_inv[['CODIGO_UNICO','FECHA_EJEC']].groupby(by=['CODIGO_UNICO']).max()
data_ejecucion_inv_2= data_ejecucion_inv[['CODIGO_UNICO','FECHA_EJEC']].groupby(by=['CODIGO_UNICO']).min()
data_ejecucion_final=data_ejecucion_inv_1.merge(right=data_ejecucion_inv_2, how="outer", on=["CODIGO_UNICO"] )


In [60]:
# join con la base de inversiones totales
data_prov_final=data_prov.merge(right=data_ejecucion_final, how="left", on=["CODIGO_UNICO"])

In [61]:
# Creando variables para el indicador
# max: FECHA_EJEC_x
# min: FECHA_EJEC_y
data_prov_final['DIF_FECHA_EJECUCION']=data_prov_final['FECHA_EJEC_x'] - data_prov_final['FECHA_EJEC_y']

# Estimando los días
data_prov_final['DIF_FECHA_EJECUCION']= data_prov_final['DIF_FECHA_EJECUCION'].dt.total_seconds()/(60*24*60)
data_prov_final['DIF_FECHA_EJECUCION']= data_prov_final['DIF_FECHA_EJECUCION'].astype(str)
data_prov_final['DIF_FECHA_EJECUCION']= data_prov_final['DIF_FECHA_EJECUCION'].astype(float)

In [62]:
# variables para el indicador
'''
El tiempo esperado de ejecución de una inversión es fijado a través de su costo de inversión (a nivel de viabilidad). De tal forma:

- Si una inversión tiene un costo de S/.1,000 millones o superior, se esperaría que se ejecute en no más de 5 años.
- Si una inversión tiene un costo entre S/. 600 millones y S/.1,000 millones, se esperaría que se ejecute en no más de 4 años
- Si una inversión tiene un costo entre S/.300 millones y S/.600 millones, se esperaría que se ejecute en no más de 3 años
- Si una inversión tiene un costo entre S/.50 millones y S/.300 millones, se esperaría que se ejecute en no más de 2 años
- Si una inversión tiene un costo menor a S/.50 millones, se esperaría que se ejecute en no más de 1 año
'''
# Numerado
data_prov_final['id_EJEC06_N']=0
data_prov_final.loc[(data_prov_final['MONTO_VIABLE']>=1000*1000000) & ((data_prov_final['DIF_FECHA_EJECUCION']>0) & (data_prov_final['DIF_FECHA_EJECUCION']<=5*365) & (data_prov_final['DEVENGADO_ACUMULADO']>0)  ),'id_EJEC06_N' ]=1
data_prov_final.loc[((data_prov_final['MONTO_VIABLE']>=600*1000000) & (data_prov_final['MONTO_VIABLE']<1000*1000000)) & ((data_prov_final['DIF_FECHA_EJECUCION']>0) & (data_prov_final['DIF_FECHA_EJECUCION']<=4*365)) & (data_prov_final['DEVENGADO_ACUMULADO']>0),'id_EJEC06_N' ]=1
data_prov_final.loc[((data_prov_final['MONTO_VIABLE']>=300*1000000) & (data_prov_final['MONTO_VIABLE']<600*1000000)) & ((data_prov_final['DIF_FECHA_EJECUCION']>0) & (data_prov_final['DIF_FECHA_EJECUCION']<=3*365)) & (data_prov_final['DEVENGADO_ACUMULADO']>0),'id_EJEC06_N' ]=1
data_prov_final.loc[((data_prov_final['MONTO_VIABLE']>=50*1000000) & (data_prov_final['MONTO_VIABLE']<300*1000000)) & ((data_prov_final['DIF_FECHA_EJECUCION']>0) & (data_prov_final['DIF_FECHA_EJECUCION']<=2*365)) & (data_prov_final['DEVENGADO_ACUMULADO']>0) ,'id_EJEC06_N' ]=1
data_prov_final.loc[((data_prov_final['MONTO_VIABLE']>0) & (data_prov_final['MONTO_VIABLE']<50*1000000)) & ((data_prov_final['DIF_FECHA_EJECUCION']>0) & (data_prov_final['DIF_FECHA_EJECUCION']<=1*365)) & (data_prov_final['DEVENGADO_ACUMULADO']>0),'id_EJEC06_N' ]=1

# Denominador
data_prov_final['id_EJEC06_D']=0
data_prov_final.loc[data_prov_final['DEVENGADO_ACUMULADO']>0,'id_EJEC06_D']=1

#Preliminar
data_prov_final_ind06=data_prov_final.copy()
data_prov_final_ind06=data_prov_final_ind06[['CODIGO_SNIP',"id_EJEC06_N","id_EJEC06_D"]]

In [63]:
#Collapsando
data_prov_final=data_prov_final[["id_ambito","ENTIDAD","id_EJEC06_N","id_EJEC06_D"]].groupby(by=['ENTIDAD', 'id_ambito'],as_index=False).sum()

In [64]:
# Estimando indicador
data_prov_final['EJEC06']=data_prov_final['id_EJEC06_N']/data_prov_final['id_EJEC06_D']

In [65]:
#Exportando resultado
#writer = pd.ExcelWriter(ruta_output / 'indicadores_7.xlsx')
data_prov_final.to_excel(writer, sheet_name='EJEC06' , index= False)
data_prov_final_6=data_prov_final.copy()
#writer.save()
#writer.close()

### **EJEC07**: Competitividad en los procesos concursales para la ejecución de inversiones


In [66]:
#Collapsando
data_seace_inv['cant_postor']=1
data_seace_inv_collap=data_seace_inv[['CODIGOCONVOCATORIA','CODIGO_UNICO', 'cant_postor']].groupby(by=['CODIGOCONVOCATORIA','CODIGO_UNICO'], as_index=False).sum()

In [67]:
#Promedio
data_seace_inv_collap=data_seace_inv_collap[['cant_postor','CODIGO_UNICO']].groupby(by=['CODIGO_UNICO'],as_index=False).mean()

In [68]:
# Identificando la cantidad de inversiones viables totales entre el 2012-2022
data_prov=data_inversiones.loc[(data_inversiones['SITUACION']=="VIABLE") | (data_inversiones['SITUACION']=="APROBADO")]     
data_prov=data_prov.drop_duplicates(['CODIGO_SNIP'], keep='first')

#Inversiones entre el 2012 y 2022
data_prov=data_prov.loc[(data_prov['año']>=2012) & (data_prov['año']<=2022)]  #Inversiones entre el 2012 y 2022

In [69]:
# join
data_prov_final=data_prov.merge(right=data_seace_inv_collap, how='left', on=['CODIGO_UNICO'])

In [70]:
# Numerador
'''
Los Puntajes de Competitividad de una comvocatoria son asignados en función al promedio de postores que una inversión tuvo por convocatoria. De tal forma:
-Si el promedio de postores es menor o igual a 1, se le asigna un puntaje de cero (0).
-Si el promedio de postores es mayor a 1 y menor que 3, se le asigna un puntaje de 0.3.
-Si el promedio de postores es mayor o igual que 3 y menor que 6, se le asigna un puntaje de 0.6.
-Si el promedio de postores es mayor o igual que 6, se le asigna un puntaje de 1.
'''
data_prov_final['id_EJEC07_N']=0
data_prov_final.loc[data_prov_final['cant_postor']<=1,'id_EJEC07_N']=0
data_prov_final.loc[((data_prov_final['cant_postor']>1) & (data_prov_final['cant_postor']<3)),'id_EJEC07_N']=0.3
data_prov_final.loc[((data_prov_final['cant_postor']>=3) & (data_prov_final['cant_postor']<6)),'id_EJEC07_N']=0.6
data_prov_final.loc[((data_prov_final['cant_postor']>=6)),'id_EJEC07_N']=1

#Denominador
data_prov_final['id_EJEC07_D']=1 #Inversiones declaradas viables entre 2012-2022

#Preliminar
data_prov_final_ind7=data_prov_final.copy()
data_prov_final_ind7=data_prov_final_ind7[['CODIGO_SNIP', "id_EJEC07_N","id_EJEC07_D"]]

In [71]:
#Collapsando
data_prov_final=data_prov_final[["id_ambito","ENTIDAD","id_EJEC07_N","id_EJEC07_D"]].groupby(by=['ENTIDAD', 'id_ambito'],as_index=False).sum()

In [72]:
# Estimando indicador
data_prov_final['EJEC07']=data_prov_final['id_EJEC07_N']/data_prov_final['id_EJEC07_D']

In [73]:
#Exportando resultado
#writer = pd.ExcelWriter(ruta_output / 'indicadores_8.xlsx')
data_prov_final.to_excel(writer, sheet_name='EJEC07' , index= False)
#writer.save()
#writer.close()

data_prov_final_7=data_prov_final.copy()


### **EJEC09**: Verificación del estado de cierre de la inversión respecto al Formato N° 9

### Se plantea el siguiente indicador: **EJEC09**: Porcentaje de inversiones que gastaron y su descripción de cierre es diferente de "No culminado"

In [74]:
# Identificando la cantidad de inversiones viables totales entre el 2012-2022
data_prov=data_inversiones.loc[(data_inversiones['SITUACION']=="VIABLE") | (data_inversiones['SITUACION']=="APROBADO")]     
data_prov=data_prov.drop_duplicates(['CODIGO_SNIP'], keep='first')

#Inversiones entre el 2012 y 2022
data_prov=data_prov.loc[(data_prov['año']>=2012) & (data_prov['año']<=2022)]  #Inversiones entre el 2012 y 2022

In [75]:
#Denominador
data_prov['id_EJEC09_D']=0 
data_prov.loc[(data_prov['DEVENGADO_ACUMULADO']>0),"id_EJEC09_D"] =1 #Inversiones con devengado mayor a cero

#Numerador
data_prov['id_EJEC09_N']=0
data_prov.loc[(data_prov['DEVENGADO_ACUMULADO']>0),"id_EJEC09_N"] =1
data_prov.loc[((data_prov['DEVENGADO_ACUMULADO']>0) & (data_prov['DES_CIERRE']=="No culminada")) ,"id_EJEC09_N"] =0 #Inversiones con devengado mayor a cero y descripcion de cierre "No culminada"

#Preliminar
data_prov_ind9=data_prov.copy()
data_prov_ind9=data_prov_ind9[['CODIGO_SNIP', "id_EJEC09_N","id_EJEC09_D"]]

In [76]:
#Collapsando
data_prov_final=data_prov[["id_ambito","ENTIDAD","id_EJEC09_N","id_EJEC09_D"]].groupby(by=['ENTIDAD', 'id_ambito'],as_index=False).sum()

In [77]:
# Estimando indicador
data_prov_final['EJEC09']=data_prov_final['id_EJEC09_N']/data_prov_final['id_EJEC09_D']

In [78]:
#Exportando resultado
#writer = pd.ExcelWriter(ruta_output / 'indicadores_8.xlsx')
data_prov_final.to_excel(writer, sheet_name='EJEC09' , index= False)
#writer.save()
#writer.close()

data_prov_final_9=data_prov_final.copy()

### **EJEC08**: Eficacia en la ejecución respecto al ET o documento equivalente

In [79]:
#Inversiones  de la etapa "Expediente técnico (B)"
data_inv_B=data_f8_detall_inv.copy()
data_inv_B=data_inv_B.loc[data_inv_B['DES_ETAPA']=='Expediente técnico (B)']

#Solo nos concentraremos en infraestructura, mobiliario, equipo, intengible, etc.
# No se considerará "GESTION DEL PROYECTO", "EXPEDIENTE TÉCNICO", "SUPERVISIÓN"
# "SUPERVISIÓN COVID", "SUPERVISIÓN COVID","GASTOS GENERALES COVID", "GESTIÓN Y OTROS NEC3-G2G ARCC"
# "ESTUDIO BASE", "GESTIÓN DEL PROGRAMA", "CONTROL CONCURRENTE"

data_inv_B=data_inv_B.loc[~((data_inv_B['DES_TIPO_COMPONENTE']=='GESTION DEL PROYECTO') | (data_inv_B['DES_TIPO_COMPONENTE']=='EXPEDIENTE TÉCNICO')
                           | (data_inv_B['DES_TIPO_COMPONENTE']=='SUPERVISIÓN') | (data_inv_B['DES_TIPO_COMPONENTE']=='SUPERVISIÓN COVID')  
                            | (data_inv_B['DES_TIPO_COMPONENTE']=='GASTOS GENERALES COVID') | (data_inv_B['DES_TIPO_COMPONENTE']=='GESTIÓN Y OTROS NEC3-G2G ARCC')
                             | (data_inv_B['DES_TIPO_COMPONENTE']=='ESTUDIO BASE') | (data_inv_B['DES_TIPO_COMPONENTE']=='GESTIÓN DEL PROGRAMA') 
                              | (data_inv_B['DES_TIPO_COMPONENTE']=='CONTROL CONCURRENTE'))]

data_inv_B['COSTO_INVERSION_B']=data_inv_B['COSTO_INVERSION']


In [80]:
data_inv_B_collapse=data_inv_B[["CODIGO_UNICO","COSTO_INVERSION_B"]].groupby(by=['CODIGO_UNICO'],as_index=False).sum()

In [81]:
#Inversiones  de la etapa "Ejecución física (C)"
data_inv_C=data_f8_detall_inv.copy()
data_inv_C=data_inv_C.loc[data_inv_C['DES_ETAPA']=='Ejecución física (C)']

#Solo nos concentraremos en infraestructura, mobiliario, equipo, intengible, etc.
# No se considerará "GESTION DEL PROYECTO", "EXPEDIENTE TÉCNICO", "SUPERVISIÓN"
# "SUPERVISIÓN COVID", "SUPERVISIÓN COVID","GASTOS GENERALES COVID", "GESTIÓN Y OTROS NEC3-G2G ARCC"
# "ESTUDIO BASE", "GESTIÓN DEL PROGRAMA", "CONTROL CONCURRENTE"

data_inv_C=data_inv_C.loc[~((data_inv_C['DES_TIPO_COMPONENTE']=='GESTION DEL PROYECTO') | (data_inv_C['DES_TIPO_COMPONENTE']=='EXPEDIENTE TÉCNICO')
                           | (data_inv_C['DES_TIPO_COMPONENTE']=='SUPERVISIÓN') | (data_inv_C['DES_TIPO_COMPONENTE']=='SUPERVISIÓN COVID')  
                            | (data_inv_C['DES_TIPO_COMPONENTE']=='GASTOS GENERALES COVID') | (data_inv_C['DES_TIPO_COMPONENTE']=='GESTIÓN Y OTROS NEC3-G2G ARCC')
                             | (data_inv_C['DES_TIPO_COMPONENTE']=='ESTUDIO BASE') | (data_inv_C['DES_TIPO_COMPONENTE']=='GESTIÓN DEL PROGRAMA') 
                              | (data_inv_C['DES_TIPO_COMPONENTE']=='CONTROL CONCURRENTE'))]

data_inv_C['COSTO_INVERSION_C']=data_inv_C['COSTO_INVERSION']


In [82]:
data_inv_C_collapse=data_inv_C[["CODIGO_UNICO","COSTO_INVERSION_C"]].groupby(by=['CODIGO_UNICO'],as_index=False).sum()

In [83]:
#join entre ambas bases
data_inv_union=data_inv_B_collapse.merge(right=data_inv_C_collapse, how='left', on=["CODIGO_UNICO"] )

In [84]:
#Generando
data_inv_union['DIFERENCIA']=data_inv_union['COSTO_INVERSION_C'] - data_inv_union['COSTO_INVERSION_B'] 
data_inv_union.loc[data_inv_union['DIFERENCIA'].isnull(), 'DIFERENCIA'] = 0


In [85]:
# Identificando la cantidad de inversiones viables totales entre el 2012-2022
data_prov=data_inversiones.loc[(data_inversiones['SITUACION']=="VIABLE")  | (data_inversiones['SITUACION']=="APROBADO")]     
data_prov=data_prov.drop_duplicates(['CODIGO_SNIP'], keep='first')

#Inversiones entre el 2012 y 2022
data_prov=data_prov.loc[(data_prov['año']>=2012) & (data_prov['año']<=2022)]  #Inversiones entre el 2012 y 2022

In [86]:
#Join
data_inv_union_vf=data_prov.merge(right=data_inv_union, how='left', on=["CODIGO_UNICO"] )

In [87]:
#Generando denominador
data_inv_union_vf['id_EJEC08_D']=1

#Generando numerador
data_inv_union_vf['id_EJEC08_N']=1
data_inv_union_vf.loc[(data_inv_union_vf['DIFERENCIA']>0),'id_EJEC08_N' ]=0
data_inv_union_vf.loc[(data_inv_union_vf['DIFERENCIA'].isnull()),'id_EJEC08_N' ]=0

#Base preliminar
data_inv_ind8=data_inv_union_vf.copy()
data_inv_ind8=data_inv_ind8[['CODIGO_SNIP',"id_EJEC08_N","id_EJEC08_D"]]

In [88]:
data_inv_union_collapse=data_inv_union_vf[["id_ambito","ENTIDAD","id_EJEC08_N","id_EJEC08_D"]].groupby(by=['ENTIDAD', 'id_ambito'],as_index=False).sum()

In [89]:
#Indicador
data_inv_union_collapse['EJEC08']=data_inv_union_collapse['id_EJEC08_N']/data_inv_union_collapse['id_EJEC08_D']

In [90]:
#Exportando resultado
#writer = pd.ExcelWriter(ruta_output / 'indicadores_6.xlsx')
data_inv_union_collapse.to_excel(writer, sheet_name='EJEC08' , index= False)
data_inv_8=data_inv_union_collapse.copy()
writer.save()
writer.close()

C:\Users\llan_\AppData\Local\Temp\ipykernel_1472\1723977604.py:5: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


### Uniendo las bases para exportar


In [91]:
# Exportando la base
writer = pd.ExcelWriter(ruta_output / 'Base_indicadores_inversion.xlsx')

#Uniendo las bases preliminares
total_base=data_prov_ind1.merge(right=data_prov_ind2_3, how="left",on=['CODIGO_SNIP'])
total_base=total_base.merge(right=data_prov_ind4, how="left",on=['CODIGO_SNIP'])
total_base=total_base.merge(right=data_prov_ind1_2_1_2_4_5, how="left",on=['CODIGO_SNIP'])
total_base=total_base.merge(right=data_et_inv_union_ind3, how="left",on=['CODIGO_SNIP'])
total_base=total_base.merge(right=data_prov_final_ind06, how="left",on=['CODIGO_SNIP'])
total_base=total_base.merge(right=data_prov_final_ind7, how="left",on=['CODIGO_SNIP'])
total_base=total_base.merge(right=data_prov_ind9, how="left",on=['CODIGO_SNIP'])
total_base=total_base.merge(right=data_inv_ind8, how="left",on=['CODIGO_SNIP'])

total_base.to_excel(writer, sheet_name='Base_indicadores_inversion' , index= False)
writer.save()
writer.close()
#total_base.to_excel(ruta_output / 'indicadores_consolidado.xlsx', sheet_name='inversiones' , index= False)


C:\Users\llan_\AppData\Local\Temp\ipykernel_1472\1414488534.py:15: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [92]:
# Exportando la base
writer = pd.ExcelWriter(ruta_output / 'Base_indicadores_final.xlsx')

#Uniendo las bases finales
total_base_final=data_prov_1.merge(right=data_prov_2_3, how="left",on=["id_ambito","ENTIDAD"])
total_base_final=total_base_final.merge(right=data_prov_union_4, how="left",on=["id_ambito","ENTIDAD"])
total_base_final=total_base_final.merge(right=data_prov_1_2_1_2_4_5, how="left",on=["id_ambito","ENTIDAD"])
total_base_final=total_base_final.merge(right=data_et_inv_collapse_3, how="left",on=["id_ambito","ENTIDAD"])
total_base_final=total_base_final.merge(right=data_prov_final_6, how="left",on=["id_ambito","ENTIDAD"])
total_base_final=total_base_final.merge(right=data_prov_final_7, how="left",on=["id_ambito","ENTIDAD"])
total_base_final=total_base_final.merge(right=data_prov_final_9, how="left",on=["id_ambito","ENTIDAD"])
total_base_final=total_base_final.merge(right=data_inv_8, how="left",on=["id_ambito","ENTIDAD"])


#total_base_final.to_excel(writer, sheet_name='Base_indicadores_all' , index= False)
#writer.save()
#writer.close()

### Indicador **GOB01**: Información revelada en los indicadores

In [93]:
# Identificando los inicadores con información
for i in range(1,16):
    total_base_final[f'id_{i}']=0

total_base_final.loc[total_base_final['PRG01_PROM'].isnull(), 'id_1']=1 
total_base_final.loc[total_base_final['PRG02'].isnull(), 'id_2']=1 
total_base_final.loc[total_base_final['PRG03'].isnull(), 'id_3']=1 
total_base_final.loc[total_base_final['PRG04'].isnull(), 'id_4']=1 
total_base_final.loc[total_base_final['FORM01'].isnull(), 'id_5']=1 
total_base_final.loc[total_base_final['FORM02'].isnull(), 'id_6']=1 
total_base_final.loc[total_base_final['EJEC01'].isnull(), 'id_7']=1 
total_base_final.loc[total_base_final['EJEC02'].isnull(), 'id_8']=1 
total_base_final.loc[total_base_final['EJEC03'].isnull(), 'id_9']=1 
total_base_final.loc[total_base_final['EJEC04'].isnull(), 'id_10']=1 
total_base_final.loc[total_base_final['EJEC05'].isnull(), 'id_11']=1 
total_base_final.loc[total_base_final['EJEC06'].isnull(), 'id_12']=1 
total_base_final.loc[total_base_final['EJEC07'].isnull(), 'id_13']=1 
total_base_final.loc[total_base_final['EJEC08'].isnull(), 'id_14']=1 
total_base_final.loc[total_base_final['EJEC09'].isnull(), 'id_15']=1 

total_base_final['id_total']=total_base_final['id_1']+total_base_final['id_2']+total_base_final['id_3']+total_base_final['id_4']+total_base_final['id_5']+total_base_final['id_6']
+total_base_final['id_7']+total_base_final['id_8']+total_base_final['id_9']+total_base_final['id_10']+total_base_final['id_11']+total_base_final['id_12']+total_base_final['id_13']
+total_base_final['id_14']+total_base_final['id_15']

total_base_final['iindicadores']=15 - total_base_final['id_total']

#Estimando el indicador de "GOB01"
total_base_final['GOB01']=total_base_final['iindicadores']/15


In [94]:
#Los promedios por Ciclo de inversión
total_base_final['SPRG']=(total_base_final['PRG01_PROM'] + total_base_final['PRG02'] + total_base_final['PRG03'] + total_base_final['PRG04']) / 4
total_base_final['SFOR']=(total_base_final['FORM01'] + total_base_final['FORM02'] ) / 2
total_base_final['SEJE']=(total_base_final['EJEC01'] + total_base_final['EJEC02'] + total_base_final['EJEC03']  + total_base_final['EJEC04']  + total_base_final['EJEC05']  + total_base_final['EJEC06'] 
                          +  total_base_final['EJEC07']  +  total_base_final['EJEC08']  +  total_base_final['EJEC09']) / 9 

In [95]:
#Indice de desempeño
total_base_final['IDGIP']=total_base_final['SPRG']*0.3+total_base_final['SFOR']*0.3+total_base_final['SEJE']*0.3+total_base_final['GOB01']*0.1

In [96]:
#Ordenando los indicadores
total_base_1=total_base_final[["id_ambito","ENTIDAD",
                  'PMI_2','PMI_2_PREVISTAS','PRG01_2',
                  'PMI_3','PMI_3_PREVISTAS','PRG01_3','PMI_4','PMI_4_PREVISTAS','PRG01_4','PRG01_PROM',
                  'MONTO_PIM','MONTO_DEVENGADO','MONTO_PROGRAMADO','PRG02','PRG03',
                  'id_brecha','id_inversion','PRG04','SPRG',
                  'id_FORM01_N','id_FORM01_D','FORM01',
                  'id_FORM02_N','id_FORM02_D','FORM02', 'SFOR',
                  'id_EJEC01_N','id_EJEC01_D','EJEC01',
                  'id_EJEC02_N','id_EJEC02_D','EJEC02',
                  'id_EJEC03_N','id_EJEC03_D','EJEC03',
                  'id_EJEC04_N','id_EJEC04_D','EJEC04',
                  'id_EJEC05_N','id_EJEC05_D','EJEC05',
                  'id_EJEC06_N','id_EJEC06_D','EJEC06',
                  'id_EJEC07_N','id_EJEC07_D','EJEC07',
                  'id_EJEC08_N','id_EJEC08_D','EJEC08',
                  'id_EJEC09_N','id_EJEC09_D','EJEC09','SEJE','GOB01','IDGIP']]

total_base_1.to_excel(writer, sheet_name='Base_indicadores_all' , index= False)

total_base_1=total_base_final[["id_ambito","ENTIDAD",'PRG01_PROM','PRG02','PRG03','PRG04','SPRG','FORM01','FORM02','SFOR','EJEC01','EJEC02',
                  'EJEC03','EJEC04','EJEC05','EJEC06','EJEC07','EJEC08','EJEC09','SEJE','GOB01','IDGIP']]

total_base_1.to_excel(writer, sheet_name='Base_indicadores_resumen' , index= False)
writer.save()
writer.close()


C:\Users\llan_\AppData\Local\Temp\ipykernel_1472\3026517890.py:25: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
